# <center>ZUM</center>
# <center> Projekt -- dokumentacja końcowa </center>

## <center> Jan Budziński -- 310609 </center>
## <center> Jarosław Nachyła -- nr_indeksu </center>

## 2. Implementacja

### 2.1. Nienadzorowana detekcja anomalii

W ramach projektu zaimplementowano klasę <i>AnomalyDetector</i>, będącą opakowaniem na algorytmy grupowania KMeans, DBSCAN oraz AgglomerativeClustering, jak również na miary niepodobieństwa euklidesową, Mahalanobisa i Manhattan. Celem tejże klasy jest stworzenie łatwego w obsłudze środowiska testowego, które w miarę możliwości (niektóre algorytmy różnią się wejściami) będzie jednorodne w użyciu dla każdego z porównywanych algorytmów i miar.
Klasa ta zawiera następujące metody:

- fit -- dopasowuje model do danych wejściowych,
- fit_predict -- wykonuje trening modelu i jednocześnie przewidująca klasy i określająca dystanse od centrów klastrów dla danych wejściowych,
- transform_distances -- określa, czy dane wejściowe są anomaliami na podstawie dystansu otrzymanego z wybranej miary niepodobieństwa,
- transform_labels -- dla liczby klastrów większych niż 2, metoda ta zmienia przypisanie do wszystkich klastrów poza najliczniejszym w anomalię.


Dzięki tym metodom użytkownik jest w stanie w łatwy sposób testować różne modele i miary niepodobieństwa, zmieniając wyłącznie jeden parametr w kodzie.

Kod implementujący tę klasę jest w pliku anomaly_detector.py.

Ponadto utworzono klasę <i>AnomalyDetectorEvaluator</i> zawierającą metody obliczające metryki potrzebne do ewaluacji wytrenowanych modeli.

Testowane metryki to:

- dokładność (accuracy)
- precyzja (precision)
- czułość (recall)
- pole pod wykresem PRC

Wszystkie te metryki zostały dokładniej opisane w dokumentacji wstępnej. Jednakże, z uwagi na fakt, iż dane z założenia są wysoce niezbalansowane (jako że są to dane anomalii, to wejścia o pozytywnej klasie stanowią poniżej 1% wszystkich) uznano, że zwykła dokładność może niewiele powiedzieć, jako że przypisanie wszystkim danym klasy negatywnej pozwala osiągnąć powyżej 99% poprawnych predykcji. W tym celu dodano metrykę dokładności wykrywania outlierów, która testuje, ile spośród prawdziwych outlierów zostało poprawnie zidentyfikowanych. 

Kod z klasą AnomalyDetectorEvaluator znajduje się w pliku metrics.py.

### 2.2. Jednoklasowy klasyfikator

## 3. Eksperymenty



### 3.1 Porównanie modeli grupowaniaz modelami klasyfikacji jednoklasowej.

Nienadzorowana detekcja anomalii na podstawie niepodobieństwa do grup wyznaczanych za pomocą algorytmów grupowania została przeprowadzona w ramach eksperymentów porównujących algorytmy klasyfikacji jednoklasowej, takie jak One-Class SVM i Isolation Forest, z algorytmami grupowania. Eksperymenty miały na celu zbadanie wydajności tych modeli w kontekście detekcji anomalii.

W ramach pierwszego eksperymentu wykonano testy porównawcze iloczynu kartezjańskiego 3 wybranych modeli, 3 miar niepodobieństwa i 2 zbiorów danych, co dało łącznie 18 pojedynczych testów.

Podczas przeprowadzania eksperymentu wystąpiły pewne problemy. Mianowicie, algorytm Agglomerative Clustering cechuje się wysokimi złożonościami: złożoność obliczeniowa to \(O(n^2)\), a czasowa to \(O(n^3)\). Z tego względu trening na całych zbiorach danych, a w szczególności na zbiorze HTTP, był niemożliwy. W związku z tym zastosowano subsampling.


### Szczegóły Subsampling i Eksperymentów

W przeprowadzonych eksperymentach dotyczących nienadzorowanej detekcji anomalii przy użyciu algorytmów grupowania, zastosowano różne techniki subsamplingu w celu obsługi dużych zbiorów danych oraz złożoności obliczeniowej niektórych algorytmów. Poniżej znajduje się szczegółowe wyjaśnienie strategii subsamplingu użytych dla różnych eksperymentów oraz wyjaśnienie, dlaczego niektóre eksperymenty nie zostały wykonane.

#### Szczegóły Subsampling

| Zbiór Danych      | Algorytm                    | Subsampling           |
|-------------------|-----------------------------|-----------------------|
| HTTP              | Isolation Forest            | 40% klasy większościowej |
| HTTP              | One-Class SVM               | 40% klasy większościowej |
| HTTP              | DBSCAN                      | 10% klasy większościowej |
| HTTP              | Agglomerative Clustering    | 5% klasy większościowej  |
| HTTP              | KMeans                      | Brak subsamplingu      |
| Shuttle           | Isolation Forest            | 40% klasy większościowej |
| Shuttle           | One-Class SVM               | 40% klasy większościowej |
| Shuttle           | DBSCAN (Mahalanobis)        | 20% klasy większościowej |
| Shuttle           | Agglomerative Clustering    | Odpowiedni subsampling w razie potrzeby |
| Shuttle           | KMeans                      | Brak subsamplingu      |



DBSCAN z metryką Mahalanobis nie zostały wykonane ze względu na bardzo długie czasy nauki nawet dla małych próbek danych(mozliwy problem w implemntacji w bibliotece).



In [30]:
import json
import os

import pandas as pd

pd.set_option('display.max_rows', 100)

def load_metrics_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data
def process_files(directory, pdataset = "http", models_excluded = []):
    results = []

    # Przeglądanie wszystkich plików w katalogu
    for file_name in os.listdir(directory):
        if file_name.endswith(".json"):
            # Parsing nazwy pliku
            parts = file_name.split('_')
            model = parts[0]
            dataset = parts[1]

            if dataset != pdataset or model in models_excluded:
                continue



            file_path = os.path.join(directory, file_name)
            metrics_data = load_metrics_from_json(file_path)



            if isinstance(metrics_data, dict):  # Dla isolation forest i svm
                metric = "N/A"
                avg_metrics = metrics_data['avg_metrics']
                result = {
                    'model': model,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'f1': avg_metrics.get('f1'),
                   
                    'auc_score': avg_metrics.get('auc_pr')
                }
                results.append(result)
            else:  # Dla pozostałych modeli
                for entry in metrics_data:
                    metric = entry['metric']
                    avg_metrics = entry['avg_metrics']
                    result = {
                    'model': model,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'f1': avg_metrics.get('f1'),
                    'auc_score': avg_metrics.get('auc_pr')
                    }
                    results.append(result)

    # Tworzenie DataFrame z wynikami
    df = pd.DataFrame(results)
    return df
pd.set_option('display.max_rows', 50)
process_files('./results/', models_excluded=['metacost'])

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,isolationforest,N/A,0.907282,1.000000,0.876376,0.729462,1.000000,0.843571,NaN
1,dbscan,euclidean,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
2,dbscan,manhattan,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
3,svm,N/A,0.623021,1.000000,0.497362,0.398738,1.000000,0.570139,NaN
4,kmeans,euclidean,0.565949,0.995477,0.564269,0.008857,0.999969,0.017557,0.538962
5,kmeans,cityblock,0.999942,0.996382,0.999956,0.988779,0.999986,0.992566,0.015064
6,kmeans,mahalanobis,0.990934,0.014925,0.994751,0.011000,0.996142,0.012666,0.538960
7,agglomerative,euclidean,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.061553
8,agglomerative,manhattan,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.063167
9,agglomerative,mahalanobis,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.062629


In [31]:
process_files('./results/', models_excluded=['metacost'])

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,isolationforest,N/A,0.907282,1.000000,0.876376,0.729462,1.000000,0.843571,NaN
1,dbscan,euclidean,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
2,dbscan,manhattan,0.993616,1.000000,0.993366,0.854988,1.000000,0.921826,0.035111
3,svm,N/A,0.623021,1.000000,0.497362,0.398738,1.000000,0.570139,NaN
4,kmeans,euclidean,0.565949,0.995477,0.564269,0.008857,0.999969,0.017557,0.538962
5,kmeans,cityblock,0.999942,0.996382,0.999956,0.988779,0.999986,0.992566,0.015064
6,kmeans,mahalanobis,0.990934,0.014925,0.994751,0.011000,0.996142,0.012666,0.538960
7,agglomerative,euclidean,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.061553
8,agglomerative,manhattan,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.063167
9,agglomerative,mahalanobis,0.994651,0.997286,0.994445,0.933531,0.999787,0.964356,0.062629


### 3.1 Porównanie algorytmu MetaCost opartego o KMeans z algorytmem KMeans


### Zbiór HTTP

In [32]:
process_files('./results/', models_excluded=['dbscan', 'agglomerative', 'isolationforest', 'svm'])

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,metacost,mahalanobis,0.580502,0.794211,0.579666,0.007331,0.998617,0.014528,NaN
1,metacost,euclidean,0.747444,0.017910,0.750297,0.036003,0.994549,0.012120,NaN
2,metacost,cityblock,0.570846,0.995025,0.569187,0.008953,0.999966,0.017747,NaN
3,kmeans,euclidean,0.565949,0.995477,0.564269,0.008857,0.999969,0.017557,0.538962
4,kmeans,cityblock,0.999942,0.996382,0.999956,0.988779,0.999986,0.992566,0.015064
5,kmeans,mahalanobis,0.990934,0.014925,0.994751,0.011000,0.996142,0.012666,0.538960


### Zbiór shuttle

In [33]:
process_files('./results/', models_excluded=['dbscan', 'agglomerative', 'isolationforest', 'svm'], pdataset='shuttle')

,model,metric,accuracy,positive_recall,negative_recall,positive_precision,negative_precision,f1,auc_score
0,metacost,mahalanobis,0.851531,0.970094,0.842399,0.321713,0.997274,0.483156,NaN
1,metacost,euclidean,0.851543,0.970664,0.842368,0.321730,0.997326,0.483265,NaN
2,metacost,cityblock,0.857625,0.957391,0.849941,0.329490,0.996154,0.490254,NaN
3,kmeans,euclidean,0.974907,0.690117,0.996841,0.943903,0.976617,0.797302,0.373327
4,kmeans,manhattan,0.786851,0.017374,0.846115,0.008621,0.917898,0.011524,0.845926
5,kmeans,mahalanobis,0.996089,0.946169,0.999934,0.999098,0.995871,0.971913,0.429757
